<a href="https://colab.research.google.com/github/subaru-shoji/yt-dlp_aivis-datasets_style-bert-vits2_colab/blob/main/aivis_dataset_and_style_bert_vits2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Parameters
url = "" # @param {type:"string"}
speaker_name = "" # @param {type:"string"}

## Aivis-Dataset

In [ ]:
# @title Install
%cd /content/
!git clone https://github.com/litagin02/Aivis-Dataset

In [ ]:
%cd /content/Aivis-Dataset

In [ ]:
!pip3 install -r requirements.txt

In [ ]:
!apt update -y
!apt install -y yt-dlp
!apt install -y libcublas11

In [ ]:
!yt-dlp --update

In [ ]:
# @title Download youtube sound as wav file.
!yt-dlp -x --audio-format wav --audio-quality 0 -o "01-Sources/{speaker_name}.%(ext)s" "{url}"

In [ ]:
!python -m Aivis create-segments --trim-silence

In [ ]:
# @title Create dataset.
!python -m Aivis create-datasets "{speaker_name}" "{speaker_name}" --accept-all

In [ ]:
# @title Delete dataset. (if you need to reset dataset.)
!rm -rf /content/Aivis-Dataset/04-Datasets/{speaker_name}

In [ ]:
# @title Download dataset.
!zip -r /content/{speaker_name}_dataset.zip /content/Aivis-Dataset/04-Datasets/{speaker_name}

# 圧縮した zip ファイルをダウンロードする
from google.colab import files
files.download(f'/content/{speaker_name}_dataset.zip')


## Style-Bert-VITS2

In [ ]:
# @title Install
%cd /content/
!git clone https://github.com/litagin02/Style-Bert-VITS2

In [ ]:
%cd /content/Style-Bert-VITS2

In [ ]:
!pip install -r requirements.txt

In [ ]:
!python initialize.py

In [ ]:
# @title Delete current dataset in google drive. (if you need to reset dataset.)
!rm -rf /content/drive/MyDrive/Style-Bert-VITS2/Data/{speaker_name}

In [ ]:
# @title Copy dataset from Aivis-Dataset.
!cp -r /content/Aivis-Dataset/04-Datasets/{speaker_name} /content/drive/MyDrive/Style-Bert-VITS2/Data/

In [ ]:
# @title Preprocess

# 上でつけたフォルダの名前`Data/{model_name}/`
model_name = speaker_name

# 学習のバッチサイズ。4ぐらいが最適か。VRAMのはみ出具合に応じて調整してください。
batch_size = 10

# 学習のエポック数（データセットを合計何周するか）。100ぐらいで十分だと思われます。
epochs = 100

# 保存頻度。何ステップごとにモデルを保存するか。分からなければデフォルトのままで。
save_every_steps = 1000

# 音声ファイルの音量を正規化するかどうか。
normalize = True

# 音声ファイルの開始・終了にある無音区間を削除するかどうか
trim = True

from webui_train import preprocess_all

preprocess_all(
    model_name=model_name,
    batch_size=batch_size,
    epochs=epochs,
    save_every_steps=save_every_steps,
    bf16_run=False, # colabの無料のやつではサポートされていないようです。
    num_processes=2,
    normalize=normalize,
    trim=trim,
)

In [ ]:
# @title Delete "Data/{speaker_name}/models". (if you need to delete models directory.)
!rm -rf /content/drive/MyDrive/Style-Bert-VITS2/Data/{speaker_name}/models

In [ ]:
# @title Train

# 上でつけたモデル名を入力。学習を途中からする場合はきちんとモデルが保存されているフォルダ名を入力。
model_name = speaker_name

import yaml
from webui_train import get_path

dataset_path, _, _, _, config_path = get_path(model_name)
with open("default_config.yml", "r", encoding="utf-8") as f:
    yml_data = yaml.safe_load(f)
yml_data["model_name"] = model_name
yml_data["dataset_path"] = dataset_path
with open("config.yml", "w", encoding="utf-8") as f:
    yaml.dump(yml_data, f, allow_unicode=True)


!python train_ms.py --config {config_path} --model {dataset_path}

In [ ]:
# @title Downalod model.

# 速度がめちゃくちゃ遅いので、直接Google Driveからダウンロードするのが早いっぽい

!zip -r /content/{speaker_name}.zip /content/drive/MyDrive/Style-Bert-VITS2/model_assets/{speaker_name}

# 圧縮した zip ファイルをダウンロードする
from google.colab import files
files.download(f'/content/{speaker_name}.zip')


In [ ]:
# @title Copy model from Google Drive to Style-Bert-VITS2 to create style.

# この処理はgoogle drive内でStyle-Bert-VITS2をcloneすればいらないんだけど、個人的にあまりgoogle drive内に直接色々置きたくはないので、コピー処理はやる
!cp -r /content/drive/MyDrive/Style-Bert-VITS2/Data/{speaker_name} /content/Style-Bert-VITS2/Data/
!cp -r /content/drive/MyDrive/Style-Bert-VITS2/model_assets/{speaker_name}/ /content/Style-Bert-VITS2/model_assets/

In [ ]:
# @title Create styles.

# 手動でshare=Trueを書き換える
!python webui_style_vectors.py

In [ ]:
# @title Launch tts web ui.
!python app.py --share --dir "/content/Style-Bert-VITS2/model_assets/"

In [ ]:
# @title Download model_assets.
!zip -r /content/model_assets.zip /content/drive/MyDrive/Style-Bert-VITS2/model_assets
!cp /content/model_assets.zip /content/drive/MyDrive/